# Globals

## You may want to play around with this

In [1]:
# v1 is faster, w/o footnote link;
# v2 slower, w/ footnote links (if possible)
API_VERSION = "v2"

## Probably not change these

In [2]:
# If True, the answer will be based only on collections in knowledge base.
# Otherwise, api will try to answer based on collections, but if it will not 
# succeed it will generate answer from the model weights themselves.
# Everything happens under the hood, user don't have to specify anything.
COLLECTIONS_ONLY = False 
API_URL = "https://api.askguru.ai"
VENDOR = "tada"
ORGANIZATION = "common"
TADA_PASSWORD = "0xdHscy8ZUvBdqQ7nC5Ef6wd"
SOURCE_PATTERN = r"\{ *'doc_idx' *: *'([^']*)'\}"
STREAM_ANSWER = True
IF_JUPYTER = True

## Collections names

In [3]:
COLLECTIONS_MAPPING = {
    "glavbuhqa": "Главбух: ответы на самые популярные вопросы за последний год",
    "glavbuharticles": "Главбух: лучшие статьи за последний год по налогам, страховым взносам и бухгалтерскому учету",
    "tgdulkarnaev": "Артур Дулкарнаев (налоговый юрист) — Telegram-канал",
    "nalog": "Налоговый кодекс Российской Федерации (НК РФ)",
    "zhilishchnyj": "Жилищный кодекс (ЖК РФ)",
    "KoAP": "Кодекс об административных правонарушениях (КоАП РФ)",
    "AO": 'Федеральный закон "Об АО"',
    "UK": "Уголовный кодекс (УК РФ)",
    "semejnyj": "Семейный кодекс (СК РФ)",
    "trud": "Трудовой кодекс (ТК РФ)",
    "tamozhnya": "Таможенный кодекс РФ от 28 мая 2003 г. N 61-ФЗ",
    "arbitr": "Арбитражный процессуальный кодекс (АПК РФ)",
    "zemlya": "Земельный кодекс (ЗК РФ)",
    "buhgalter": 'Федеральный закон "О бухгалтерском учете"',
    "grazhdan": "Гражданский кодекс Российской Федерации (ГК РФ)",
    "grad": "Градостроительный кодекс РФ (ГрК РФ) от 29 декабря 2004 г. N 190-ФЗ",
    "strahovanie": 'Федеральный закон от 29 декабря 2006 г. N 255-ФЗ "Об обязательном социальном страховании на случай временной нетрудоспособности и в связи с материнством" (с изменениями и дополнениями)',
    "grazhdanoprocesualny": "Гражданский процессуальный кодекс (ГПК РФ)",
    "konstitucia": "Конституция Российской Федерации",
    "personalnyedannye": 'Закон "О персональных данных"',
    "obrazovanie": "Закон об образовании",
    "voennyi": 'Федеральный закон "О воинской обязанности и военной службе"',
    "ohranazdorovia": 'Федеральный закон от 21 ноября 2011 г. N 323-ФЗ "Об основах охраны здоровья граждан в Российской Федерации" (с изменениями и дополнениями)',
    "potrebytely": 'Закон "О защите прав потребителей"',
    "goszakup": 'Закон о контрактной системе (закон о госзакупках). Федеральный закон от 5 апреля 2013 г. N 44-ФЗ "О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд" (с изменениями и дополнениями)',
    "profilaktika": 'Федеральный закон от 24 июня 1999 г. N 120-ФЗ "Об основах системы профилактики безнадзорности и правонарушений несовершеннолетних" (с изменениями и дополнениями)',
    "pubvlast": 'Федеральный закон от 21 декабря 2021 г. N 414-ФЗ "Об общих принципах организации публичной власти в субъектах Российской Федерации" (с изменениями и дополнениями)',
}

## Imports

In [ ]:
! pip install pandas loguru

In [4]:
import requests
import json
from loguru import logger
import re
import pandas as pd
from pprint import pprint

if IF_JUPYTER:
    from IPython.display import Markdown, display, clear_output

# Api definition

These are just API calls with fancy results displaying

In [5]:
class Api:
    def __init__(self, api_url: str, api_version: str, source_pattern: str, collections_mapping: dict = {}, if_jupyter: bool = False):
        self.api_url = api_url
        self.api_version = api_version
        self.token = None
        self.if_jupyter = if_jupyter
        self.source_pattern = source_pattern
        self.collections_mapping = collections_mapping
        self.collections = []

    def _make_request(
        self, method: str, endpoint: str, params: dict = {}, json: dict = {}
    ):
        url = f"{self.api_url}/{self.api_version}{endpoint}"
        response = requests.request(
            method,
            url,
            params=params,
            json=json,
            headers={"Authorization": f"Bearer {self.token}"},
            stream=params.get("stream", False),
        )
        if response.status_code != 200:
            raise Exception(f"Error: {response.status_code} {response.text}")

        return response

    def info(self):
        response = self._make_request("GET", "/info").json()
        logger.info(f"Token info: {response}")

    def authenticate(self, vendor: str, organization: str, password: str):
        json = {"vendor": vendor, "organization": organization, "password": password}
        response = self._make_request("POST", "/collections/token", json=json).json()
        self.token = response["access_token"]
        logger.info(f"Authenticated!")

    def get_collections(self):
        response = self._make_request("GET", "/collections").json()
        text = ""
        for collection in response["collections"]:
            text += f"- `{collection['name']}` ({collection['n_documents']} документов)"
            text += f" — {self.collections_mapping[collection['name']]}" if collection['name'] in self.collections_mapping else ""
            text += "\n"
        display(Markdown(text))
        self.collections = [collection['name'] for collection in response["collections"]]
        return response["collections"]

    def upload_collection_documents(self, collection: str, documents: list):
        json = {"documents": documents}
        response = self._make_request("POST", f"/collections/{collection}", json=json).json()
        logger.info(f"Uploaded documents: {response}")
    
    def get_ranking(self, query: str, collections: list[str] = [], top_k: int = 5):
        if not collections:
            collections = self.collections
        params = {"query": query, "collections": collections, "top_k": top_k}
        response = self._make_request("GET", "/collections/ranking", params=params).json()
        text = "Самые релевантные документы:\n"
        for i, source in enumerate(response["sources"]):
            text += f"{i + 1}. [{source['title'].replace('message text: ', '')}]({source['id']})"
            # text += f" ({source['summary'].split('>')[0]})" if '>' in source['summary'] else ""
            text += f" ({self.collections_mapping[source['collection']]})" if source['collection'] in self.collections_mapping else ""
            text += "\n"
        display(Markdown(text))
        return response["sources"]
    
    @staticmethod
    def postprocess_output(s: str) -> str:
        return s[: max(s.find(")"), s.rfind("."), s.rfind("?"), s.rfind("!"), s.rfind("\n")) + 1]

    def get_answer(self, query: str, user: str = "", collections: list[str] = [], collections_only: bool = False, stream: bool = False):
        if not collections:
            collections = self.collections
        params = {"collections": collections, "query": query, "user": user, "collections_only": collections_only, "stream": stream}
        response = self._make_request("GET", f"/collections/answer", params=params)
        if stream:
            answer = ""
            generated_sources = []
            for line in response.iter_lines():
                if line.startswith(b'event: '):
                    event = line[len(b'event: '):].decode()
                elif line.startswith(b'data: '):
                    data_str = line[len(b'data: '):].decode()
                    data = json.loads(data_str)
                    if event == 'message':
                        sources = data['sources']
                        request_id = data['request_id']
                        answer += data['answer']
                        match = re.findall(self.source_pattern, answer)
                        if match:
                            idx = int(match[0])
                            source = sources[idx]
                            if source in generated_sources:
                                num = generated_sources.index(source) + 1
                            else:
                                generated_sources.append(source)
                                num = len(generated_sources)
                            answer = re.sub(self.source_pattern, f"[[{num}]]({source['id']})", answer)
                        if self.if_jupyter:
                            clear_output(wait=True)
                            display(Markdown(answer))
                        else:
                            print(answer, end='\r')
            answer = Api.postprocess_output(answer)
            clear_output(wait=True)
            display(Markdown(answer))
            if generated_sources:
                answer += "\n\n**Источники:**\n"
                for i, source in enumerate(generated_sources):
                    answer += f"{i + 1}. [{source['title'].replace('message text: ', '')}]({source['id']})\n"
                clear_output(wait=True)
                display(Markdown(answer))

            response = {"answer": answer, "sources": sources, "request_id": request_id}
        return response
    
    def set_reaction(self, request_id: str, rating: int):
        json = {"request_id": request_id, "rating": rating}
        response = self._make_request("POST", "/reactions", json=json)
        logger.info(response)
        
    def get_reactions(self):
        response = self._make_request("GET", "/reactions").json()
        response = pd.DataFrame(response["reactions"])
        response.sort_values(by="datetime", ascending=False, inplace=True)
        return response
    
    def get_filters(self):
        response = self._make_request("GET", "/filters").json()
        return response
        
    def upload_filter(self, name: str, stop_words: list[str], description: str = None):
        json = {"name": name, "description": description, "stop_words": stop_words}
        response = self._make_request("POST", "/filters", json=json)
        logger.info(response)
        
    def patch_filter(self, name: str, stop_words: list[str], description: str = None):
        json = {"name": name, "description": description, "stop_words": stop_words}
        response = self._make_request("PATCH", "/filters", json=json)
        logger.info(response)
        
    def archive_filter(self, name: str):
        json = name
        response = self._make_request("DELETE", "/filters", json=json)
        logger.info(response)

In [6]:
api = Api(api_url=API_URL, api_version=API_VERSION, source_pattern=SOURCE_PATTERN, collections_mapping=COLLECTIONS_MAPPING, if_jupyter=IF_JUPYTER)

## Retrieving token

In [7]:
api.authenticate(VENDOR, ORGANIZATION, TADA_PASSWORD)
api.info()

2023-05-28 15:56:04.120 | INFO     | __main__:authenticate:36 - Authenticated!
2023-05-28 15:56:04.689 | INFO     | __main__:info:30 - Token info: {'vendor': 'tada', 'organization': 'common', 'security_groups': []}


## Getting collections

In [8]:
collections = api.get_collections()

- `UK` (1080 документов) — Уголовный кодекс (УК РФ)
- `buhgalter` (31 документов) — Федеральный закон "О бухгалтерском учете"
- `goszakup` (799 документов) — Закон о контрактной системе (закон о госзакупках). Федеральный закон от 5 апреля 2013 г. N 44-ФЗ "О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд" (с изменениями и дополнениями)
- `glavbuhqa` (516 документов) — Главбух: ответы на самые популярные вопросы за последний год
- `grazhdanoprocesualny` (729 документов) — Гражданский процессуальный кодекс (ГПК РФ)
- `glavbuharticles` (3734 документов) — Главбух: лучшие статьи за последний год по налогам, страховым взносам и бухгалтерскому учету
- `zhilishchnyj` (686 документов) — Жилищный кодекс (ЖК РФ)
- `arbitr` (761 документов) — Арбитражный процессуальный кодекс (АПК РФ)
- `strahovanie` (137 документов) — Федеральный закон от 29 декабря 2006 г. N 255-ФЗ "Об обязательном социальном страховании на случай временной нетрудоспособности и в связи с материнством" (с изменениями и дополнениями)
- `ohranazdorovia` (402 документов) — Федеральный закон от 21 ноября 2011 г. N 323-ФЗ "Об основах охраны здоровья граждан в Российской Федерации" (с изменениями и дополнениями)
- `zemlya` (657 документов) — Земельный кодекс (ЗК РФ)
- `grazhdan` (2071 документов) — Гражданский кодекс Российской Федерации (ГК РФ)
- `personalnyedannye` (95 документов) — Закон "О персональных данных"
- `obrazovanie` (541 документов) — Закон об образовании
- `grad` (1088 документов) — Градостроительный кодекс РФ (ГрК РФ) от 29 декабря 2004 г. N 190-ФЗ
- `konstitucia` (186 документов) — Конституция Российской Федерации
- `potrebytely` (116 документов) — Закон "О защите прав потребителей"
- `profilaktika` (117 документов) — Федеральный закон от 24 июня 1999 г. N 120-ФЗ "Об основах системы профилактики безнадзорности и правонарушений несовершеннолетних" (с изменениями и дополнениями)
- `pubvlast` (274 документов) — Федеральный закон от 21 декабря 2021 г. N 414-ФЗ "Об общих принципах организации публичной власти в субъектах Российской Федерации" (с изменениями и дополнениями)
- `KoAP` (2752 документов) — Кодекс об административных правонарушениях (КоАП РФ)
- `tgdulkarnaev` (503 документов) — Артур Дулкарнаев (налоговый юрист) — Telegram-канал
- `trud` (930 документов) — Трудовой кодекс (ТК РФ)
- `tamozhnya` (1 документов) — Таможенный кодекс РФ от 28 мая 2003 г. N 61-ФЗ
- `AO` (420 документов) — Федеральный закон "Об АО"
- `semejnyj` (239 документов) — Семейный кодекс (СК РФ)
- `nalog` (5969 документов) — Налоговый кодекс Российской Федерации (НК РФ)
- `voennyi` (282 документов) — Федеральный закон "О воинской обязанности и военной службе"


# Getting answers!

Play around here, ask your own questions.

You may want to specify collections to search in, but by default all available collections will be used.

You may use `api.get_ranking` to retrieve most relevant docs although it is not mandatory for the answer.

Note that API requests for questions which refer to existing knowledge and which use internal knowledge are the same.
The model under the hood decides if it is appropriate to use knowledge from knowledge base.

## Questions which refer to existing knowledge

In [9]:
# you may specify any user identifier here
USER = "322"

### Glavbuh QA

In [10]:
query = "НДФЛ с иностранных граждан временно пребывающих на патенте, перечислять нужно отдельным платежом, как и в ФСС НС?"
sources = api.get_ranking(query)

Самые релевантные документы:
1. [НДФЛ с иностранных граждан временно пребывающих на патенте](https://www.glavbukh.ru/hl/2041692-ndfl-s-inostrannyh-grajdan-vremenno-prebyvayushchih-na-patente) (Главбух: ответы на самые популярные вопросы за последний год)
2. [Как дистанционному сотруднику платить российский НДФЛ и надо ли](https://t.me/arturdulkarnaev) (Артур Дулкарнаев (налоговый юрист) — Telegram-канал)
3. [РБК пишет, что у переехавших за границу и ставших нерезидентами есть вариант зарегистрироваться самозанятыми. Я считаю, что это сработает только если вы не будете получать оплату по договору от нынешнего или бывшего работодателя](https://t.me/arturdulkarnaev) (Артур Дулкарнаев (налоговый юрист) — Telegram-канал)
4. [НДФЛ с нерезидентов в 2023 году | Ставка НДФЛ для иностранных граждан](https://www.glavbukh.ru/art/91466-ndfl-s-nerezidentov-v-2023-godu-stavka-ndfl-dlya-inostrannyh-grajdan) (Главбух: лучшие статьи за последний год по налогам, страховым взносам и бухгалтерскому учету)
5. [❓У меня часто спрашивают, мол, российский ИП на УСН с налоговой ставкой 6 процентов уехал в Армению и прожил там больше 183 дней в 2022 году. Надо ли теперь такому ИП пересчитать налоги и начать платить НДФЛ по ставке 30 процентов? Деятельность идет, деньги приходят на российский счет](https://t.me/arturdulkarnaev) (Артур Дулкарнаев (налоговый юрист) — Telegram-канал)


In [11]:
response = api.get_answer(query, user=USER, collections_only=COLLECTIONS_ONLY, stream=STREAM_ANSWER)
api.set_reaction(request_id=response["request_id"], rating=5)

Да, НДФЛ с доходов иностранных граждан временно пребывающих на патенте нужно перечислять отдельным платежным поручением. Это осуществляется самостоятельно и не включается в единый налоговый платеж[[1]](https://www.glavbukh.ru/hl/2041692-ndfl-s-inostrannyh-grajdan-vremenno-prebyvayushchih-na-patente).

**Источники:**
1. [НДФЛ с иностранных граждан временно пребывающих на патенте](https://www.glavbukh.ru/hl/2041692-ndfl-s-inostrannyh-grajdan-vremenno-prebyvayushchih-na-patente)


2023-05-28 15:56:27.260 | INFO     | __main__:set_reaction:121 - <Response [200]>


### Glavbuh Articles

In [12]:
query = "Новые правила по отпускным 2023"
sources = api.get_ranking(query)

Самые релевантные документы:
1. [Новые правила предоставления отпусков в 2023 году: разъяснения](https://www.glavbukh.ru/art/386195-novye-pravila-predostavleniya-otpuskov-v-2023-godu-razyasneniya) (Главбух: лучшие статьи за последний год по налогам, страховым взносам и бухгалтерскому учету)
2. [Новые правила предоставления отпусков в 2023 году](https://www.glavbukh.ru/art/388043-novye-pravila-predostavleniya-otpuskov-v-2023-godu) (Главбух: лучшие статьи за последний год по налогам, страховым взносам и бухгалтерскому учету)
3. [Заявления на отпуск: образцы 2023 года](https://www.glavbukh.ru/art/386331-zayavleniya-na-otpusk-obraztsy-2023-goda) (Главбух: лучшие статьи за последний год по налогам, страховым взносам и бухгалтерскому учету)
4. [Изменится срок выплаты отпускных в 2023 году](https://www.glavbukh.ru/art/388102-izmenitsya-srok-vyplaty-otpusknyh-v-2023-godu) (Главбух: лучшие статьи за последний год по налогам, страховым взносам и бухгалтерскому учету)
5. [Когда выгодно брать отпуск в 2023 году по деньгам: таблица по месяцам](https://www.glavbukh.ru/art/386238-kogda-vygodno-brat-otpusk-v-2023-godu-po-dengam-tablitsa-po-mesyatsam) (Главбух: лучшие статьи за последний год по налогам, страховым взносам и бухгалтерскому учету)


In [13]:
response = api.get_answer(query, user=USER, collections_only=COLLECTIONS_ONLY, stream=STREAM_ANSWER)
api.set_reaction(request_id=response["request_id"], rating=4)

В 2023 году утверждены новые правила предоставления отпусков. Изменения в Трудовом кодексе расширили категории работников, которые могут требовать предоставления отпуска в любое время. Вот некоторые ключевые изменения:

1. Сотрудникам с трех или более детьми до 18 лет предоставляется отпуск в удобное для них время до достижения младшим из детей 14 лет или одному из старших – 18 лет. Как только младшему исполнилось 14 или одному из старших – 18, сотрудник теряет право на льготу [[1]](https://www.glavbukh.ru/art/386195-novye-pravila-predostavleniya-otpuskov-v-2023-godu-razyasneniya).

2. Сотрудникам, призванным на службу в рамках частичной мобилизации, предоставляется отпуск в любое удобное время в течение 6-ти месяцев с момента восстановления на работе (ст. 351.7 ТК) [[2]](https://www.glavbukh.ru/art/388043-novye-pravila-predostavleniya-otpuskov-v-2023-godu).

Также планируются изменения в сроке выдачи отпускных, установленном в статье 136 ТК. Госдума рассматривает проект, который касается ситуации, когда сотрудник подает заявление на отпуск за три дня или меньше до его начала. Для этого случая предусмотрят два срока в зависимости от способа выплаты денег [[1]](https://www.glavbukh.ru/art/386195-novye-pravila-predostavleniya-otpuskov-v-2023-godu-razyasneniya).



**Источники:**
1. [Новые правила предоставления отпусков в 2023 году: разъяснения](https://www.glavbukh.ru/art/386195-novye-pravila-predostavleniya-otpuskov-v-2023-godu-razyasneniya)
2. [Новые правила предоставления отпусков в 2023 году](https://www.glavbukh.ru/art/388043-novye-pravila-predostavleniya-otpuskov-v-2023-godu)


2023-05-28 15:57:54.606 | INFO     | __main__:set_reaction:121 - <Response [200]>


### Telegram @dulkarnaev

In [14]:
query = "штраф за неподачу отчета о движении денег"
sources = api.get_ranking(query)

Самые релевантные документы:
1. [Максимальный штраф для физических лиц, если не подать отчет о движении денег и иных финансовых активов или подать такой отчет позже срока или подать с ошибками, составляет 3 тысячи рублей](https://t.me/arturdulkarnaev) (Артур Дулкарнаев (налоговый юрист) — Telegram-канал)
2. [Штрафы в сфере валютного законодательства](https://t.me/arturdulkarnaev) (Артур Дулкарнаев (налоговый юрист) — Telegram-канал)
3. [Тезисы про декларирование зарубежных счетов 2/2](https://t.me/arturdulkarnaev) (Артур Дулкарнаев (налоговый юрист) — Telegram-канал)
4. [Есть два случая, когда можно не подавать отчет о движении денег и иных финансовых активов за 2022 год. На мой взгляд, лучше подать отчет, даже если вы по закону можете его не подавать](https://t.me/arturdulkarnaev) (Артур Дулкарнаев (налоговый юрист) — Telegram-канал)
5. [Штраф за незаконную валютную операцию можно снизить на 50 процентов, если оплатить штраф в течение 20 дней после того, как налоговая вас оштрафует](https://t.me/arturdulkarnaev) (Артур Дулкарнаев (налоговый юрист) — Telegram-канал)


In [15]:
response = api.get_answer(query, user=USER, collections_only=COLLECTIONS_ONLY, stream=STREAM_ANSWER)
api.set_reaction(request_id=response["request_id"], rating=5)

Штраф за неподачу отчета о движении денег и иных финансовых активов для физических лиц составляет от 2 000 до 3 000 рублей. Если повторно не подать отчет, то штраф составит 20 тысяч рублей [[1]](https://t.me/arturdulkarnaev).

**Источники:**
1. [Максимальный штраф для физических лиц, если не подать отчет о движении денег и иных финансовых активов или подать такой отчет позже срока или подать с ошибками, составляет 3 тысячи рублей](https://t.me/arturdulkarnaev)


2023-05-28 15:58:20.290 | INFO     | __main__:set_reaction:121 - <Response [200]>


### Taxes

In [16]:
query = "налоговые вычеты для студентов многодетных семей нк рф, коротко"
sources = api.get_ranking(query)

Самые релевантные документы:
1. [Статья 218. Стандартные налоговые вычеты](https://base.garant.ru/10900200/1458a0a55ea2ab5e42fde990265bcdf4/) (Налоговый кодекс Российской Федерации (НК РФ))
2. [Статья 219. Социальные налоговые вычеты](https://base.garant.ru/10900200/2c2d4c47652499da777b2c19de85035c/) (Налоговый кодекс Российской Федерации (НК РФ))
3. [Статья 45. Исполнение обязанности по уплате налога, сбора, страховых взносов](https://base.garant.ru/10900200/c7f0164139c159e5c4e7786790ae469d/) (Налоговый кодекс Российской Федерации (НК РФ))
4. [Все налоги с зарплаты в 2023 году в процентах: таблица, примеры](https://www.glavbukh.ru/art/387852-vse-nalogi-s-zarplaty-v-2023-godu-v-protsentah-tablitsa-primery) (Главбух: лучшие статьи за последний год по налогам, страховым взносам и бухгалтерскому учету)
5. [Статья 7. Международные договоры по вопросам налогообложения](https://base.garant.ru/10900200/e88847e78ccd9fdb54482c7fa15982bf/) (Налоговый кодекс Российской Федерации (НК РФ))


In [17]:
response = api.get_answer(query, user=USER, collections_only=COLLECTIONS_ONLY, stream=STREAM_ANSWER)
api.set_reaction(request_id=response["request_id"], rating=5)

Стандартный налоговый вычет предоставляется на каждого ребенка в возрасте до 18 лет, а также на каждого студента в возрасте до 24 лет, обучающегося по очной форме. Вычет составляет 6 000 рублей на каждого ребенка. Для единственного родителя (приемного родителя), усыновителя, опекуна, попечителя вычет предоставляется в двойном размере. Социальный налоговый вычет предоставляется налогоплательщиком-родителем за обучение своих детей до 24 лет в размере фактически произведенных расходов на обучение, но не более 50 000 рублей на каждого ребенка в общей сумме на обоих родителей [[1]](https://base.garant.ru/10900200/1458a0a55ea2ab5e42fde990265bcdf4/) [[2]](https://base.garant.ru/10900200/2c2d4c47652499da777b2c19de85035c/).

**Источники:**
1. [Статья 218. Стандартные налоговые вычеты](https://base.garant.ru/10900200/1458a0a55ea2ab5e42fde990265bcdf4/)
2. [Статья 219. Социальные налоговые вычеты](https://base.garant.ru/10900200/2c2d4c47652499da777b2c19de85035c/)


2023-05-28 15:59:09.246 | INFO     | __main__:set_reaction:121 - <Response [200]>


### Labor

In [18]:
query = "Исключается ли из оплаты сверхурочных работ работы, выполненные работником в выходной день?"
sources = api.get_ranking(query)

Самые релевантные документы:
1. [Оформление и оплата работы в выходные и праздничные дни по ТК РФ в 2023 году](https://www.glavbukh.ru/art/100641-oformlenie-i-oplata-raboty-v-vyhodnye-i-prazdnichnye-dni-po-tk-rf-v-2023-godu) (Главбух: лучшие статьи за последний год по налогам, страховым взносам и бухгалтерскому учету)
2. [Норма времени при расчете праздничных часов для расчета тарифной ставки](https://www.glavbukh.ru/hl/2030084-norma-vremeni-pri-raschete-prazdnichnyh-chasov-dlya-rascheta-tarifnoy-stavki) (Главбух: ответы на самые популярные вопросы за последний год)
3. [31 декабря 2022 года выходной или рабочий день](https://www.glavbukh.ru/art/100634-31-dekabrya-2022-goda-vyhodnoy-ili-rabochiy-den) (Главбух: лучшие статьи за последний год по налогам, страховым взносам и бухгалтерскому учету)
4. [Почему лучше выбрать двойную оплату за работу в праздник](https://www.glavbukh.ru/art/387261-pochemu-luchshe-vybrat-dvoynuyu-oplatu-za-rabotu-v-prazdnik) (Главбух: лучшие статьи за последний год по налогам, страховым взносам и бухгалтерскому учету)
5. [1 сентября 2022 года выходной или рабочий день в России: последние новости](https://www.glavbukh.ru/art/387759-1-sentyabrya-2022-goda-vyhodnoy-ili-rabochiy-den-v-rossii-poslednie-novosti) (Главбух: лучшие статьи за последний год по налогам, страховым взносам и бухгалтерскому учету)


In [19]:
response = api.get_answer(query, user=USER, collections_only=COLLECTIONS_ONLY, stream=STREAM_ANSWER)
api.set_reaction(request_id=response["request_id"], rating=5)

Согласно статье 153 ТК, работу в выходной или праздничный день оплачивайте исходя из часовой или дневной ставки сверх оклада. Если сотрудник переработал месячную норму, рассчитайте доплату за работу в выходной или праздничный день. Таким образом, оплата за работу в выходной день не исключается из оплаты сверхурочных работ, а оплачивается отдельно с учетом условий, указанных в Трудовом кодексе и внутренних документах организации [[1]](https://www.glavbukh.ru/hl/2030084-norma-vremeni-pri-raschete-prazdnichnyh-chasov-dlya-rascheta-tarifnoy-stavki).

**Источники:**
1. [Норма времени при расчете праздничных часов для расчета тарифной ставки](https://www.glavbukh.ru/hl/2030084-norma-vremeni-pri-raschete-prazdnichnyh-chasov-dlya-rascheta-tarifnoy-stavki)


2023-05-28 15:59:47.157 | INFO     | __main__:set_reaction:121 - <Response [200]>


## Common knowledge questions

### EBITDA

In [20]:
query = "как расчитывается ebitda"
response = api.get_answer(query, user=USER, collections_only=COLLECTIONS_ONLY, stream=STREAM_ANSWER)
api.set_reaction(request_id=response["request_id"], rating=5)

EBITDA (Earnings Before Interest, Taxes, Depreciation, and Amortization) - это показатель прибыли компании до учета процентов, налогов, амортизации и износа. EBITDA используется для анализа финансовой эффективности компании и сравнения с другими компаниями в отрасли.

Расчет EBITDA производится по следующей формуле:

EBITDA = Чистая прибыль + Проценты + Налоги + Амортизация + Износ

1. Чистая прибыль - это доходы компании минус расходы, не включая проценты, налоги, амортизацию и износ.
2. Проценты - это платежи по процентам по кредитам и займам.
3. Налоги - это налог на прибыль, который компания обязана уплатить государству.
4. Амортизация - это систематическое распределение стоимости долгосрочных активов компании на протяжении их срока службы.
5. Износ - это учет снижения стоимости основных средств в результате их использования и старения.

Подсчитав все эти компоненты и применив формулу, вы получите значение EBITDA для анализа финансовой эффективности компании.

2023-05-28 16:00:57.268 | INFO     | __main__:set_reaction:121 - <Response [200]>


### Design pricing

In [21]:
query = "какова средняя стоимость дизайнера в час на рынке"
response = api.get_answer(query, user=USER, collections_only=COLLECTIONS_ONLY, stream=STREAM_ANSWER)
api.set_reaction(request_id=response["request_id"], rating=4)

Средняя стоимость дизайнера в час на рынке может варьироваться в зависимости от опыта, местоположения и специализации дизайнера. В среднем, стоимость услуг дизайнера в час может колебаться от 500 до 2500 рублей. Опытные дизайнеры или те, кто специализируется на определенных областях, могут взимать более высокую плату. Также стоит учитывать, что цены могут отличаться в зависимости от региона и спроса на услуги дизайнера.

2023-05-28 16:01:35.324 | INFO     | __main__:set_reaction:121 - <Response [200]>


# Retrieving logs with feedback

In [22]:
reactions = api.get_reactions()
display(reactions)

,id,datetime,query,answer,api_version,collections,user,rating,like_status,comment
172,64730a41602c5c7bcd4c076e,2023-05-28T08:01:05.786000,какова средняя стоимость дизайнера в час на рынке,Средняя стоимость дизайнера в час на рынке мож...,v2,"[UK, buhgalter, goszakup, glavbuhqa, grazhdano...",322,4.0,None,None
171,647309fb602c5c7bcd4c076d,2023-05-28T07:59:55.948000,как расчитывается ebitda,"EBITDA (Earnings Before Interest, Taxes, Depre...",v2,"[UK, buhgalter, goszakup, glavbuhqa, grazhdano...",322,5.0,None,None
170,647309d8602c5c7bcd4c076c,2023-05-28T07:59:20.439000,Исключается ли из оплаты сверхурочных работ ра...,"Согласно статье 153 ТК, работу в выходной или ...",v2,"[UK, buhgalter, goszakup, glavbuhqa, grazhdano...",322,5.0,None,None
169,647309a7602c5c7bcd4c076b,2023-05-28T07:58:31.531000,налоговые вычеты для студентов многодетных сем...,Стандартный налоговый вычет предоставляется на...,v2,"[UK, buhgalter, goszakup, glavbuhqa, grazhdano...",322,5.0,None,None
168,6473098e602c5c7bcd4c076a,2023-05-28T07:58:06.369000,штраф за неподачу отчета о движении денег,Штраф за неподачу отчета о движении денег и ин...,v2,"[UK, buhgalter, goszakup, glavbuhqa, grazhdano...",322,5.0,None,None
...,...,...,...,...,...,...,...,...,...,...
4,646bab2435733c1c08f83bc7,2023-05-22T17:49:24.955000,как заполнить декларацию,,v1,[nalog],None,NaN,None,None
3,646baaed35733c1c08f83bc6,2023-05-22T17:48:29.899000,как заполнить декларацию,,v1,[nalog],None,NaN,None,None
2,646baad435733c1c08f83bc5,2023-05-22T17:48:04.992000,как заполнить декларацию,,v1,[nalog],None,NaN,None,None
1,646ba9d935733c1c08f83bc4,2023-05-22T17:43:53.703000,как заполнить декларацию,Для заполнения налоговой декларации необходимо...,v1,[nalog],None,NaN,None,None


# Filtering rules

We do not send separate response that request has made it through filters and the answer is preparing.

The reason for this is that we stream answer, so if you have not received an error right away, you will immediately receive an event stream with 200 status code, so there is no need in separate response.

To every filter rule you may add description.

In [38]:
pprint(api.get_filters())

{'active_rules': [{'description': None,
                   'name': 'religion',
                   'stop_words': ['вера'],
                   'timestamp': 1685261263},
                  {'description': None,
                   'name': 'politics',
                   'stop_words': [],
                   'timestamp': 1685261270}],
 'archived_rules': []}


In [39]:
query = "что значит мы русские, с нами бог"

In [40]:
# 'бог' is not in ban
response = api.get_answer(query, user=USER, collections_only=COLLECTIONS_ONLY, stream=STREAM_ANSWER)

"Мы русские, с нами Бог" - это фраза, которая олицетворяет гордость и уверенность в себе русского народа. Она подразумевает, что русские люди считают себя защищенными и благословленными Богом, что придает им силу и мужество. Это выражение часто используется в контексте патриотизма и национальной идентичности.

In [41]:
api.patch_filter(name="religion", stop_words=["бог", "вера"])
# 'бог' is in ban, so we are getting an error
response = api.get_answer(query, user=USER, collections_only=COLLECTIONS_ONLY, stream=STREAM_ANSWER)

2023-05-28 16:08:43.645 | INFO     | __main__:patch_filter:141 - <Response [200]>


Exception: Error: 422 {"detail":"Request blocked by organization policy rule: 'religion' (None)"}

In [42]:
api.archive_filter(name="religion")
# as rule is archived, we are able to query once again
response = api.get_answer(query, user=USER, collections_only=COLLECTIONS_ONLY, stream=STREAM_ANSWER)

"Мы русские, с нами Бог" - это фраза, которая подчеркивает национальную идентичность и веру в божественную защиту. В переводе на английский язык это означает "We are Russians, God is with us". Фраза может использоваться для выражения гордости за свою страну и уверенности в том, что Бог будет поддерживать и защищать свой народ.

In [43]:
# we can upload any new filter
# api.upload_filter(name="cars", stop_words=["девятка", "жигули"])

In [44]:
# patching archived filter leads to it resurrection
api.patch_filter(name="religion", stop_words=["вера"])

2023-05-28 16:09:20.892 | INFO     | __main__:patch_filter:141 - <Response [200]>
